#### Step 1: Initial data inspection

- Load dataset into a dataframe with Pandas
- Understand the structure of each dataset, including columns and data types
- Identify data types and missing values (noted potential mixed data types)

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re

In [2]:
# load datasets
cb_data = pd.read_csv('C:/Users/AM/OneDrive/Desktop/capstone/data/raw/space_startups.csv', encoding='latin-1')
usa_spending = pd.read_csv('C:/Users/AM/OneDrive/Desktop/capstone/data/raw/prime_awards.csv', encoding='utf-8')

In [3]:
# check data shape
print("Crunchbase Data Shape:", cb_data.shape)
print("USA Spending Data Shape:", usa_spending.shape)

Crunchbase Data Shape: (904, 24)
USA Spending Data Shape: (5294, 60)


This difference in size and complexity suggests that we'll need to carefully select which USA Spending data to merge with the Crunchbase data.

In [4]:
# display the first few rows of each dataset
print("\nCrunchbase Data:")
print(cb_data.head())
print("\nUSA Spending Data:")
print(usa_spending.head())


Crunchbase Data:
           company_name  year_founded  \
0                DIMAAG          2018   
1               Netdata          2018   
2  Outward Technologies          2017   
3                TATINE          2015   
4                   X3M          2022   

                                          industries  \
0  Artificial Intelligence (AI), Autonomous Vehic...   
1  Enterprise Software, Information Technology, I...   
2                       Aerospace, Commercial, Solar   
3   E-Commerce, Home Decor, Manufacturing, Wholesale   
4  Advertising, Advertising Platforms, Informatio...   

                                         hq  zipcode  \
0        Fremont, California, United States    94538   
1  San Francisco, California, United States    94104   
2       Broomfield, Colorado, United States    80020   
3          Chicago, Illinois, United States    60639   
4       Wilmington, Delaware, United States    19801   

                                         description operatin

In [5]:
# check for missing values
print("\nMissing values in Crunchbase Data:")
print(cb_data.isnull().sum())
print("\nMissing values in USA Spending Data:")
print(usa_spending.isnull().sum())


Missing values in Crunchbase Data:
company_name            0
year_founded            0
industries              0
hq                      0
zipcode                 0
description             0
operating_status        0
cb_rank                 0
funding_status        527
funding_date          482
funding_type          426
acquired_by           862
acquired_price        899
ipo_status              0
ipo_valuation         898
ipo_date              894
growth_confidence     903
growth_category       739
estimated_revenue     394
num_employees          25
num_funding_rounds    426
total_funding         503
top_investors         516
num_investors         517
dtype: int64

Missing values in USA Spending Data:
contract_award_unique_key                            0
award_id_piid                                        0
parent_award_agency_id                            2959
parent_award_agency_name                          3047
parent_award_id_piid                              2959
total_obligate

**Crunchbase Data**: many fields in this dataset have missing values. We will drop unnecessary rows.
<br>**USA Spending Data**: Many columns have missing values, with some having over 4,000 missing entries out of 5,294 rows.

<br>**Action items**:
<br>We need to identify which columns are crucial for our analysis and focus on cleaning those. Some columns with many missing values might be less relevant or could be dropped.

In [6]:
# Crunchbase data cleaning
columns_to_drop_cb = ['acquired_by', 'acquired_price', 'ipo_valuation', 'ipo_date', 'growth_confidence', 'growth_category']
cb_data = cb_data.drop(columns=columns_to_drop_cb)

# drop rows with missing critical information
cb_data = cb_data.dropna(subset=['company_name', 'year_founded', 'Industries', 'hq', 'description', 'operating_status'])

In [6]:
# USA Spending data cleaning
columns_to_drop_usa = [
    'parent_award_agency_id', 'parent_award_agency_name', 'parent_award_id_piid',
    'total_outlayed_amount', 'treasury_accounts_funding_this_award',
    'object_classes_funding_this_award', 'program_activities_funding_this_award', 'recipient_name_raw',
    'recipient_address_line_2', 'prime_award_summary_recipient_county_fips_code',
    'recipient_county_name', 'prime_award_summary_recipient_state_fips_code',
    'other_than_full_and_open_competition'
]
usa_spending = usa_spending.drop(columns=columns_to_drop_usa)

# zero rows with missing critical information

In [7]:
# check data types
print("\nData types in Crunchbase Data:")
print(cb_data.dtypes)
print("\nData types in USA Spending Data:")
print(usa_spending.dtypes)


Data types in Crunchbase Data:
company_name           object
year_founded          float64
Industries             object
hq                     object
zipcode               float64
description            object
operating_status       object
cb_rank               float64
funding_status         object
funding_date           object
funding_type           object
ipo_status             object
estimated_revenue      object
num_employees          object
num_funding_rounds    float64
total_funding         float64
top_investors          object
num_investors         float64
dtype: object

Data types in USA Spending Data:
contract_award_unique_key                          object
award_id_piid                                      object
parent_award_agency_id                             object
parent_award_agency_name                           object
parent_award_id_piid                               object
total_obligated_amount                            float64
total_outlayed_amount           

**Crunchbase Data**: Many columns are of type 'object', which suggests they're all being treated as strings.
<br>**USA Spending Data**: Mix of 'object' and 'float64' types.

**Action Items**:
<br>For Crunchbase, we should convert numeric columns (e.g., funding amounts, employee counts) to appropriate numeric types.
<br>For USA Spending, we need to ensure date columns are in datetime format and numeric columns are in appropriate numeric types.


In [9]:
# ensure 'year_founded' is integer
cb_data['year_founded'] = cb_data['year_founded'].astype(int)

In [10]:
# convert operating_status to binary (saves space)
# 'Active' will be '1', 'Closed' will be '0'
cb_data['operating_status_binary'] = (cb_data['operating_status'] == 'Active').astype(int)

In [32]:
# function to correct and categorize num_employees
def correct_employee_category(value):
    if value == '10-Jan':  # Fix Excel conversion issue
        return '1-10'
    elif pd.isna(value):
        return 'Unknown'
    else:
        return value

In [33]:
# apply the correction
cb_data['num_employees_cat'] = cb_data['num_employees_cat'].apply(correct_employee_category)

In [35]:
# define the order of categories
employee_categories = ['1-10', '11-50', '51-100', '101-250', '251-500', '501-1000', '1001-5000', '10001+', 'Unknown']

# apply the correction and convert to categorical
cb_data['num_employees_cat'] = (cb_data['num_employees']
                                .apply(categorize_employees)
                                .astype(pd.CategoricalDtype(categories=employee_categories, ordered=True)))

In [36]:
# verify the changes
print(cb_data['num_employees_cat'].value_counts())

num_employees_cat
1-10         420
11-50        319
51-100        52
101-250       52
251-500       13
501-1000      12
1001-5000      5
10001+         5
Unknown        0
Name: count, dtype: int64


#### Step 2: Crunchbase data cleaning

- Handle missing values in key fields like 'funding_date', 'total_funding'
- Convert date fields to a consistent format
- Standardize company names (remove LLC, Inc., etc.)
- Create a standardized location column from 'hq'
- Process funding information
- Categorize companies based on industry focus

In [18]:
# function to safely replace NaN with 0
def safe_fillna(col):
    if pd.api.types.is_categorical_dtype(col):
        # for categorical columns, we'll add '0' to the categories if it's not already there
        if '0' not in col.cat.categories:
            col = col.cat.add_categories('0')
        return col.fillna('0')
    else:
        return col.fillna(0)

# apply safe_fillna to all columns
cb_data = cb_data.apply(safe_fillna)

In [24]:
# standardize company names
def clean_company_name(name):
    name = str(name)
    name = re.sub(r'\s+(LLC|Inc\.?|Corporation|Corp\.?|Ltd\.?)$', '', name, flags=re.IGNORECASE)
    return name.upper().strip()

cb_data['clean_name'] = cb_data['company_name'].apply(clean_company_name)

In [14]:
# extract and clean location data
cb_data[['city', 'state', 'country']] = cb_data['hq'].str.split(',', expand=True)
cb_data['state'] = cb_data['state'].str.strip()
cb_data['country'] = cb_data['country'].str.strip()

In [25]:
# convert specified columns to integers
int_columns = ['cb_rank', 'num_funding_rounds', 'num_investors', 'total_funding']

for col in int_columns:
    cb_data[col] = cb_data[col].astype(float).fillna(0).astype(int)

In [26]:
# verify the changes
print(cb_data[int_columns].dtypes)

cb_rank               int32
num_funding_rounds    int32
num_investors         int32
total_funding         int32
dtype: object


In [37]:
# save the cleaned data
cb_data.to_csv('cleaned_crunchbase_data.csv', index=False)

In [29]:
print(cb_data.head())
print(cb_data.info())

           company_name  year_founded  \
0                DIMAAG          2018   
1               Netdata          2018   
2  Outward Technologies          2017   
3                TATINE          2015   
4                   X3M          2022   

                                          Industries  \
0  Artificial Intelligence (AI), Autonomous Vehic...   
1  Enterprise Software, Information Technology, I...   
2                       Aerospace, Commercial, Solar   
3   E-Commerce, Home Decor, Manufacturing, Wholesale   
4  Advertising, Advertising Platforms, Informatio...   

                                         hq  zipcode  \
0        Fremont, California, United States    94538   
1  San Francisco, California, United States    94104   
2       Broomfield, Colorado, United States    80020   
3          Chicago, Illinois, United States    60639   
4       Wilmington, Delaware, United States    19801   

                                         description operating_status  \
0  Bui

**Output Analysis**:

1. Dataset Overview:
   - We have 904 companies in the dataset.
   - The data spans various aspects of each company, including founding date, industry, location, funding, and operational status.

2. Data Quality:
   - All columns have full data (904 non-null values), which is good.

3. Key Observations:
   - Funding amounts vary significantly, from $35.8 million (Phase Four) to $1.36 billion (Relativity Space).
   - The number of investors ranges from 11 to 58 for these companies, indicating varying levels of investor interest.

4. Data Types:
   - Most columns (30) are of type 'object', which may include strings and mixed data types.
   - 'founded_year' type is appropriate for data visualization.
   - categories were added for easy range comparison
   - 'total_funding', 'num_funding_rounds' are int types, suitable for numerical analysis.


#### Step 3: USA Spending data cleaning

- Focus on the 'Contracts_PrimeAwardSummaries' file as it seems most relevant
- Standardize company names in 'prime_awardee_name'
- Convert date fields to a consistent format
- Filter out irrelevant awards (e.g., those related to malnutrition)
- Create a year column from 'prime_award_base_action_date'
- Categorize awards based on 'prime_award_naics_description'

In [7]:
# let's check again which column we are dealing with
print("Current columns in USA Spending data:")
for col in usa_spending.columns:
    print(col)

print(f"\nTotal number of columns: {len(usa_spending.columns)}")

Current columns in USA Spending data:
contract_award_unique_key
award_id_piid
total_obligated_amount
current_total_value_of_award
potential_total_value_of_award
award_base_action_date
award_base_action_year
award_latest_action_date
current_end_date
awarding_agency_code
awarding_agency_name
awarding_sub_agency_code
awarding_sub_agency_name
awarding_office_code
awarding_office_name
funding_agency_code
funding_agency_name
funding_sub_agency_code
funding_sub_agency_name
funding_office_code
funding_office_name
recipient_uei
recipient_name
cage_code
recipient_parent_uei
recipient_parent_name
recipient_parent_name_raw
recipient_address_line_1
recipient_city_name
recipient_state_code
recipient_zip_4_code
recipient_phone_number
primary_place_of_performance_city_name
primary_place_of_performance_state_code
primary_place_of_performance_zip_4
award_type
type_of_contract_pricing
product_or_service_code_description
naics_code
naics_description
solicitation_procedures
number_of_offers_received
perfor

In [8]:
columns_to_keep = [
    'contract_award_unique_key', 'award_id_piid', 'total_obligated_amount',
    'current_total_value_of_award', 'potential_total_value_of_award',
    'award_base_action_date', 'award_base_action_year', 'current_end_date',
    'awarding_agency_name', 'awarding_sub_agency_name',
    'funding_agency_name', 'funding_sub_agency_name',
    'recipient_name', 'recipient_parent_name',
    'recipient_city_name', 'recipient_state_code', 'recipient_zip_4_code',
    'primary_place_of_performance_city_name', 'primary_place_of_performance_state_code',
    'award_type', 'type_of_contract_pricing',
    'product_or_service_code_description', 'naics_code', 'naics_description',
    'number_of_offers_received', 'organizational_type'
]

usa_spending = usa_spending[columns_to_keep]

print(f"Columns remaining: {usa_spending.shape[1]}")
print(usa_spending.columns.tolist())

Columns remaining: 26
['contract_award_unique_key', 'award_id_piid', 'total_obligated_amount', 'current_total_value_of_award', 'potential_total_value_of_award', 'award_base_action_date', 'award_base_action_year', 'current_end_date', 'awarding_agency_name', 'awarding_sub_agency_name', 'funding_agency_name', 'funding_sub_agency_name', 'recipient_name', 'recipient_parent_name', 'recipient_city_name', 'recipient_state_code', 'recipient_zip_4_code', 'primary_place_of_performance_city_name', 'primary_place_of_performance_state_code', 'award_type', 'type_of_contract_pricing', 'product_or_service_code_description', 'naics_code', 'naics_description', 'number_of_offers_received', 'organizational_type']


In [9]:
# convert date columns to datetime
date_columns = ['award_base_action_date', 'current_end_date']
for col in date_columns:
    usa_spending[col] = pd.to_datetime(usa_spending[col], errors='coerce')

In [10]:
# convert numeric columns
numeric_columns = ['total_obligated_amount', 'current_total_value_of_award', 'potential_total_value_of_award', 'award_base_action_year', 'number_of_offers_received', 'naics_code']
for col in numeric_columns:
    usa_spending[col] = pd.to_numeric(usa_spending[col], errors='coerce')

In [11]:
# fill missing values for numeric columns with 0
usa_spending[numeric_columns] = usa_spending[numeric_columns].fillna(0)

In [12]:
# fill missing values for categorical columns with 'Unknown'
categorical_columns = [col for col in usa_spending.columns if col not in numeric_columns + date_columns]
usa_spending[categorical_columns] = usa_spending[categorical_columns].fillna('Unknown')

In [13]:
# standardize company names for potential matching with CB data
def clean_company_name(name):
    if pd.isna(name):
        return ''
    name = str(name)
    # remove commas
    name = name.replace(',', '')
    # remove common suffixes
    name = re.sub(r'\s+(LLC|Inc\.?|Corporation|Corp\.?|Ltd\.?|Limited|Company|Co\.?|L\.?P\.?|S\.?A\.?)$', '', name, flags=re.IGNORECASE)
    # remove any remaining punctuation and convert to uppercase
    name = re.sub(r'[^\w\s]', '', name)
    # remove extra spaces and convert to uppercase
    return ' '.join(word.upper() for word in name.split() if word.isalnum())

In [14]:
# apply to USA Spending data
usa_spending['clean_recipient_name'] = usa_spending['recipient_name'].apply(clean_company_name)
usa_spending['clean_recipient_parent_name'] = usa_spending['recipient_parent_name'].apply(clean_company_name)

In [15]:
# display a few examples from USA Spending data to verify the cleaning
print("USA Spending Data:")
print(usa_spending[['recipient_name', 'clean_recipient_name', 'recipient_parent_name', 'clean_recipient_parent_name']].head())

USA Spending Data:
                              recipient_name  \
0                  THE AEROSPACE CORPORATION   
1                     SPIRIT AEROSYSTEMS INC   
2                            MAXAR SPACE LLC   
3  THE REGENTS OF THE UNIVERSITY OF COLORADO   
4        BAE SYSTEMS LAND AND ARMAMENTS L.P.   

                        clean_recipient_name  \
0                              THE AEROSPACE   
1                         SPIRIT AEROSYSTEMS   
2                                MAXAR SPACE   
3  THE REGENTS OF THE UNIVERSITY OF COLORADO   
4             BAE SYSTEMS LAND AND ARMAMENTS   

                       recipient_parent_name  \
0                  THE AEROSPACE CORPORATION   
1          SPIRIT AEROSYSTEMS HOLDINGS, INC.   
2                            MAXAR SPACE LLC   
3  THE REGENTS OF THE UNIVERSITY OF COLORADO   
4   COMPAGNIE DE DEVELOPPEMENT DE L'EAU S.A.   

                 clean_recipient_parent_name  
0                              THE AEROSPACE  
1                SPI

In [16]:
# display the updated data types
print(usa_spending.dtypes)

contract_award_unique_key                          object
award_id_piid                                      object
total_obligated_amount                            float64
current_total_value_of_award                      float64
potential_total_value_of_award                    float64
award_base_action_date                     datetime64[ns]
award_base_action_year                              int64
current_end_date                           datetime64[ns]
awarding_agency_name                               object
awarding_sub_agency_name                           object
funding_agency_name                                object
funding_sub_agency_name                            object
recipient_name                                     object
recipient_parent_name                              object
recipient_city_name                                object
recipient_state_code                               object
recipient_zip_4_code                               object
primary_place_

In [17]:
# convert NAICS code to string
usa_spending['naics_code'] = usa_spending['naics_code'].astype(str).str.zfill(6)

In [18]:
# convert number_of_offers_received to nullable integer
usa_spending['number_of_offers_received'] = usa_spending['number_of_offers_received'].astype('Int64')

In [19]:
# fixing zipcodes
def standardize_zip(zip_code):
    zip_str = str(zip_code).zfill(5)  # pad to 5 digits
    
    if len(zip_str) >= 5:
        return zip_str[:5]  # return first 5 digits for 9-digit (ZIP+4) codes
    elif len(zip_str) == 5:
        return zip_str  # return as-is for correct 5-digit codes
    else:
        return '00000'  # default to '00000' for any other case

In [20]:
# apply to relevant column
zip_column = ['recipient_zip_4_code']

for col in zip_column:
    usa_spending[col] = usa_spending[col].apply(standardize_zip)

In [23]:
# rename the column
usa_spending = usa_spending.rename(columns={'recipient_zip_4_code': 'recipient_zipcode'})

In [25]:
# check the results
print(usa_spending.columns)

Index(['contract_award_unique_key', 'award_id_piid', 'total_obligated_amount',
       'current_total_value_of_award', 'potential_total_value_of_award',
       'award_base_action_date', 'award_base_action_year', 'current_end_date',
       'awarding_agency_name', 'awarding_sub_agency_name',
       'funding_agency_name', 'funding_sub_agency_name', 'recipient_name',
       'recipient_parent_name', 'recipient_city_name', 'recipient_state_code',
       'recipient_zipcode', 'primary_place_of_performance_city_name',
       'primary_place_of_performance_state_code', 'award_type',
       'type_of_contract_pricing', 'product_or_service_code_description',
       'naics_code', 'naics_description', 'number_of_offers_received',
       'organizational_type', 'clean_recipient_name',
       'clean_recipient_parent_name'],
      dtype='object')


In [26]:
# count the frequency of ZIP code lengths after standardization
for col in ['recipient_zipcode']:
    print(f"\nZIP code lengths for {col}:")
    print(usa_spending[col].str.len().value_counts())


ZIP code lengths for recipient_zipcode:
recipient_zipcode
5    5294
Name: count, dtype: int64


In [27]:
# print the head of the entire table
print(usa_spending.head().to_string())

                        contract_award_unique_key  award_id_piid  total_obligated_amount  current_total_value_of_award  potential_total_value_of_award award_base_action_date  award_base_action_year current_end_date                           awarding_agency_name                       awarding_sub_agency_name                            funding_agency_name                        funding_sub_agency_name                             recipient_name                      recipient_parent_name recipient_city_name recipient_state_code recipient_zipcode primary_place_of_performance_city_name primary_place_of_performance_state_code           award_type type_of_contract_pricing                                                         product_or_service_code_description naics_code                                                                                                   naics_description  number_of_offers_received       organizational_type                       clean_recipient_name             

In [28]:
# check the updated data types
print("\nUpdated Data Types:")
print(usa_spending.dtypes)


Updated Data Types:
contract_award_unique_key                          object
award_id_piid                                      object
total_obligated_amount                            float64
current_total_value_of_award                      float64
potential_total_value_of_award                    float64
award_base_action_date                     datetime64[ns]
award_base_action_year                              int64
current_end_date                           datetime64[ns]
awarding_agency_name                               object
awarding_sub_agency_name                           object
funding_agency_name                                object
funding_sub_agency_name                            object
recipient_name                                     object
recipient_parent_name                              object
recipient_city_name                                object
recipient_state_code                               object
recipient_zipcode                                  

**Output Analysis**:

- **ZIP codes**: The `recipient_zipcode` column is now of type 'object', which is appropriate for preserving leading zeros in ZIP codes.
- **Date columns**: `award_base_action_date` and `current_end_date` are in datetime64[ns] format, which is perfect for date-based analysis.
- **Numeric columns**: `total_obligated_amount`, `current_total_value_of_award`, and `potential_total_value_of_award` are float64, which is appropriate for monetary values.
- **Integer columns**: `award_base_action_year` is int64, and `number_of_offers_received` is Int64 (nullable integer).
- **Categorical columns**: Most other columns are of type 'object', which is suitable for categorical data.
- **Clean company names**: The `clean_recipient_name` and `clean_recipient_parent_name` columns have been added, which will be useful for matching with the Crunchbase data. We will keep the original column for reference.
- **NAICS code**: The `naics_code` column is now of type 'object', which preserves any leading zeros in the codes.

We can proceed with saving our results.

In [29]:
# save the cleaned data
usa_spending.to_csv('cleaned_usa_spending_data.csv', index=False)